***INSTALL REQUIREMENTS***

In [1]:
# %pip install ultralytics==8.2.103 -q
# %pip install roboflow --quiet
# %pip install supervision==0.24.0 -q
# %pip install face_recognition opencv-python numpy
# %pip install face_recognition

***IMPORT STATION***

In [1]:
from ultralytics import YOLO
from IPython.display import display, Image
from IPython.display import clear_output
from datetime import datetime
from flask import Flask, render_template, Response
import yaml
import supervision as sv
import ultralytics
import cv2
import sqlite3
import os
import atexit
import numpy as np
import face_recognition
import time

# Path model dan database
MODEL_PATH = "runs/obb/train/weights/best.pt"
DB_PATH = "presensi.db"

***TRAINING***

In [ ]:
!yolo settings sync=False

ultralytics.checks()

dataset = "/home/abe-tanu/Documents/Code/Python/Artificial-Intelligence/OBB AI PPTI 21 Team 2.v2i.yolov8-obb/datasets"

# Download the dataset
with open(f'{dataset}/data.yaml', 'r') as f:
    data = yaml.safe_load(f)
data['train'] = '../train/images'
data['val'] = '../valid/images'
data['test'] = '../test/images'
if 'path' in data:
  del data['path']
with open(f'{dataset}/data.yaml', 'w') as f:
    yaml.dump(data, f, sort_keys=False)

# Train the model
model = YOLO('yolov8n-obb.pt')
results = model.train(data=f"{dataset}/data.yaml", epochs=100, imgsz=640)


***MODELING***

**Utility Function**

In [2]:
# Koneksi global
conn = sqlite3.connect(DB_PATH, check_same_thread=False)
c = conn.cursor()

# Buat tabel jika belum ada
c.execute('''
    CREATE TABLE IF NOT EXISTS presensi (
        nama TEXT,
        status TEXT,
        waktu TEXT
    )
''')
conn.commit()

def sudah_terdeteksi(nama):
    c.execute("SELECT 1 FROM presensi WHERE nama=? AND status='Hadir'", (nama,))
    return c.fetchone() is not None

def tandai_hadir(nama):
    waktu = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    c.execute("INSERT INTO presensi (nama, status, waktu) VALUES (?, 'Hadir', ?)", (nama, waktu))
    conn.commit()

def get_status_presensi(daftar_anak):
    c.execute("SELECT nama, waktu FROM presensi WHERE status = 'Hadir'")
    hasil = c.fetchall()
    nama_hadir = set(nama for nama, _ in hasil)
    tidak_hadir = [nama for nama in daftar_anak if nama not in nama_hadir]
    return hasil, tidak_hadir

def reset_presensi():
    c.execute("DELETE FROM presensi")
    conn.commit()
    print("📛 Semua data presensi berhasil dihapus.")


**Load Face Encoding**

In [3]:
def load_known_faces(folder="faces"):
    known_face_encodings = []
    known_face_names = []

    for filename in os.listdir(folder):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            path = os.path.join(folder, filename)
            image = face_recognition.load_image_file(path)
            encodings = face_recognition.face_encodings(image)
            if encodings:
                known_face_encodings.append(encodings[0])
                name = os.path.splitext(filename)[0]
                known_face_names.append(name)
            else:
                print(f"[WARNING] Tidak ditemukan wajah di {filename}")
    return known_face_encodings, known_face_names

# Panggil jika ingin muat wajah
# known_face_encodings, known_face_names = load_known_faces()


**Setup Yolo**

In [4]:
# Daftar anak-anak
daftar_anak = ["Adriel Bernhard T", "Jonea Kristiawan", "Kevin Tanwiputra", "Kevin Jiovanni Kuslin"]

# Load YOLOv8-OBB model
model = YOLO(MODEL_PATH)

***Flask***

**Streaming Flask**

In [6]:
def gen():
    cap = cv2.VideoCapture(0)

    while True:
        success, frame = cap.read()
        if not success:
            break

        results = model.predict(source=frame, conf=0.75, stream=True)

        for r in results:
            if r.obb is not None and hasattr(r.obb, "xywhr"):
                cls_list = r.obb.cls.cpu().numpy().astype(int)
                names = model.names

                for cls in cls_list:
                    nama_terdeteksi = names[cls] if cls in names else f"Unknown-{cls}"
                    if nama_terdeteksi in daftar_anak:
                        if not sudah_terdeteksi(nama_terdeteksi):
                            tandai_hadir(nama_terdeteksi)
                            print(f"[INFO] {nama_terdeteksi} TERDETEKSI - disimpan ke database")

            # Tambah overlay
            annotated_frame = r.plot()
            cv2.rectangle(annotated_frame, (10, 10), (500, 10 + 30 + 25 * len(daftar_anak)), (0, 0, 0), -1)
            cv2.putText(annotated_frame, "Presensi Siswa", (20, 35), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

            y_offset = 60
            for nama in sorted(daftar_anak):
                status = "Hadir" if sudah_terdeteksi(nama) else "Belum Hadir"
                warna = (0, 255, 0) if status == "Hadir" else (0, 0, 255)
                cv2.putText(annotated_frame, f"{nama} - {status}", (20, y_offset),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.55, warna, 2)
                y_offset += 25

            _, buffer = cv2.imencode('.jpg', annotated_frame)
            frame = buffer.tobytes()
            yield (b'--frame\r\n'
                   b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')

    cap.release()


**Run Flask**

In [7]:
app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')  # Harus buat file ini di folder templates/

@app.route('/video_feed')
def video_feed():
    return Response(gen(), mimetype='multipart/x-mixed-replace; boundary=frame')

@app.route('/rekap')
def rekap():
    hadir, tidak_hadir = get_status_presensi(daftar_anak)
    return render_template("rekap.html", hadir=hadir, tidak_hadir=tidak_hadir)

# Jalankan Flask (non-blocking dalam Jupyter perlu trik khusus)
app.run(host='0.0.0.0', port=5000, debug=True, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://10.68.109.20:5000
Press CTRL+C to quit
127.0.0.1 - - [14/Jul/2025 16:15:20] "GET / HTTP/1.1" 200 -



0: 480x640 116.7ms


127.0.0.1 - - [14/Jul/2025 16:15:21] "GET /video_feed HTTP/1.1" 200 -


Speed: 1.8ms preprocess, 116.7ms inference, 1.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 111.5ms
Speed: 1.7ms preprocess, 111.5ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)



127.0.0.1 - - [14/Jul/2025 16:15:21] "GET /favicon.ico HTTP/1.1" 404 -


0: 480x640 111.0ms
Speed: 3.3ms preprocess, 111.0ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 94.7ms
Speed: 1.1ms preprocess, 94.7ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 77.2ms
Speed: 1.7ms preprocess, 77.2ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 81.9ms
Speed: 1.6ms preprocess, 81.9ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 104.2ms
Speed: 0.8ms preprocess, 104.2ms inference, 1.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 85.0ms
Speed: 1.8ms preprocess, 85.0ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 74.0ms
Speed: 1.5ms preprocess, 74.0ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 118.4ms
Speed: 3.4ms preprocess, 118.4ms inference, 1.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 67.1ms
Speed: 1.2ms preprocess, 67.1ms inference, 2.4ms